In [1]:
import pandas as pd
import altair as alt
import io
from IPython.display import display, HTML

In [2]:
class Edge():
    def __init__(self, source_level, source_node_id, destination_level, destination_node_id, weight):
        self.source_level = source_level #'level_001'
        self.source_node_id = source_node_id # 'node_001'
        self.destination_level = destination_level #'level_002'
        self.destination_node_id = destination_node_id #'node_001'
        self.weight = weight #50

In [3]:
edge1 = Edge(
    source_level = 1,
    source_node_id = 'node_001',
    destination_level = 2,
    destination_node_id = 'node_001',
    weight = 30
)

edge2 = Edge(
    source_level = 1,
    source_node_id = 'node_001',
    destination_level = 2,
    destination_node_id = 'node_002',
    weight = 70
)

edge3 = Edge(
    source_level = 1,
    source_node_id = 'node_002',
    destination_level = 2,
    destination_node_id = 'node_001',
    weight = 77
)

edge4 = Edge(
    source_level = 1,
    source_node_id = 'node_002',
    destination_level = 2,
    destination_node_id = 'node_002',
    weight = 88
)

edge5 = Edge(
    source_level = 2,
    source_node_id = 'node_001',
    destination_level = 3,
    destination_node_id = 'node_001',
    weight = 107
)

edge6 = Edge(
    source_level = 2,
    source_node_id = 'node_002',
    destination_level = 3,
    destination_node_id = 'node_002',
    weight = 158
)

edge7 = Edge(
    source_level = 1,
    source_node_id = 'node_001',
    destination_level = 2,
    destination_node_id = 'node_003',
    weight = 20
)

edges = []
edges.append(edge1)
edges.append(edge2)
edges.append(edge3)
edges.append(edge4)
edges.append(edge5)
edges.append(edge6)
edges.append(edge7)

In [4]:
source_nodes = {}

for edge in edges:
    if not edge.source_level in source_nodes:
        source_nodes[edge.source_level] = {}

    if not edge.source_node_id in source_nodes[edge.source_level]:
        source_nodes[edge.source_level][edge.source_node_id] = 0

    source_nodes[edge.source_level][edge.source_node_id] += edge.weight

source_nodes

{1: {'node_001': 120, 'node_002': 165}, 2: {'node_001': 107, 'node_002': 158}}

In [5]:
destination_nodes = {}

for edge in edges:
    if not edge.destination_level in destination_nodes:
        destination_nodes[edge.destination_level] = {}

    if not edge.destination_node_id in destination_nodes[edge.destination_level]:
        destination_nodes[edge.destination_level][edge.destination_node_id] = 0

    destination_nodes[edge.destination_level][edge.destination_node_id] += edge.weight

destination_nodes

{2: {'node_001': 107, 'node_002': 158, 'node_003': 20},
 3: {'node_001': 107, 'node_002': 158}}

In [6]:
nodes = source_nodes

for level in destination_nodes:
    if level not in source_nodes:
        nodes[level] = destination_nodes[level]
        continue
    if level in source_nodes:
        for node in destination_nodes[level]:
            if node not in source_nodes[level]:
                nodes[level][node] = destination_nodes[level][node]


In [7]:
nodes_pdf = pd.DataFrame(
    [(level, node, weight) for level, nodes in nodes.items() for node, weight in nodes.items()],
    columns=['level', 'node_id', 'weight']
)

gap = 10
cumsum_with_padding = (
    nodes_pdf.groupby('level')['weight']
        .apply(lambda x: (x + gap).cumsum() - gap)
        .reset_index(level=0, drop=True)
)
nodes_pdf['node_y2'] = -cumsum_with_padding

nodes_pdf['node_y1'] = nodes_pdf['node_y2'] + nodes_pdf['weight']

nodes_pdf['out_pointer'] = nodes_pdf['node_y1']
nodes_pdf['in_pointer'] = nodes_pdf['node_y1']

In [8]:
def update_pointer(level, node_id, weight, direction):
    idx = nodes_pdf.index[
        (nodes_pdf.level == level)
        & (nodes_pdf.node_id == node_id)
    ].tolist()[0]

    if (direction=='in'):
        nodes_pdf.at[idx, 'in_pointer'] = nodes_pdf.at[idx, 'in_pointer'] - weight
        
    if (direction=='out'):
        nodes_pdf.at[idx, 'out_pointer'] = nodes_pdf.at[idx, 'out_pointer'] - weight


def get_pointer(level, node_id, direction):
    idx = nodes_pdf.index[
        (nodes_pdf.level == level)
        & (nodes_pdf.node_id == node_id)
    ].tolist()[0]

    if (direction=='in'):
        return nodes_pdf.at[idx, 'in_pointer']
        
    if (direction=='out'):
        return nodes_pdf.at[idx, 'out_pointer']


In [9]:
edge_ids = []
levels = []
edge_y1s = []
edge_y2s = []
edge_weights = []

for i, edge in enumerate(edges):
    edge_ids += [f'edge_{i:03}']*6
    edge_weights += [edge.weight]*6

    edge_source_y1 = get_pointer(edge.source_level, edge.source_node_id, 'out')
    edge_destination_y1 = get_pointer(edge.destination_level, edge.destination_node_id, 'in')

    edge_mid1_y1 = edge_source_y1
    edge_mid2_y1 = edge_source_y1
    edge_mid3_y1 = edge_source_y1

    edge_mid4_y1 = edge_destination_y1
    edge_mid5_y1 = edge_destination_y1
    edge_mid6_y1 = edge_destination_y1

    
     
    curve_offset_ratio1 = 0.01
    curve_offset_ratio2 = 0.05
    '''
    if (edge_source_y1 < edge_destination_y1):
        edge_mid2_y1 += edge.weight * curve_offset_ratio1
        edge_mid3_y1 += edge.weight * curve_offset_ratio2

        edge_mid4_y1 -= edge.weight * curve_offset_ratio2
        edge_mid5_y1 -= edge.weight * curve_offset_ratio1
    elif (edge_source_y1 > edge_destination_y1):
        edge_mid2_y1 -= edge.weight * curve_offset_ratio1
        edge_mid3_y1 -= edge.weight * curve_offset_ratio2

        edge_mid4_y1 += edge.weight * curve_offset_ratio2
        edge_mid5_y1 += edge.weight * curve_offset_ratio1
    else:
        pass
    '''
    mid_level_offset = 0.05

    levels += [edge.source_level]
    edge_y1s += [edge_source_y1]
    edge_y2s += [edge_source_y1-edge.weight]
    update_pointer(level=edge.source_level, node_id=edge.source_node_id, weight=edge.weight, direction='out')

    levels += [edge.source_level+mid_level_offset]
    edge_y1s += [edge_mid1_y1]
    edge_y2s += [edge_mid1_y1-edge.weight]

    levels += [edge.source_level+mid_level_offset*2]
    edge_y1s += [edge_mid2_y1]
    edge_y2s += [edge_mid2_y1-edge.weight]

    #levels += [edge.source_level+mid_level_offset*6]
    #edge_y1s += [edge_mid3_y1-edge.weight*0.1]
    #edge_y2s += [edge_mid3_y1-edge.weight*1.1]



    #levels += [edge.destination_level-mid_level_offset*6]
    #edge_y1s += [edge_mid4_y1+edge.weight*0.3]
    #edge_y2s += [edge_mid4_y1-edge.weight*0.8]

    levels += [edge.destination_level-mid_level_offset*2]
    edge_y1s += [edge_mid5_y1]
    edge_y2s += [edge_mid5_y1-edge.weight]

    levels += [edge.destination_level-mid_level_offset]
    edge_y1s += [edge_mid6_y1]
    edge_y2s += [edge_mid6_y1-edge.weight]

    levels += [edge.destination_level]
    edge_y1s += [edge_destination_y1]
    edge_y2s += [edge_destination_y1-edge.weight]
    update_pointer(level=edge.destination_level, node_id=edge.destination_node_id, weight=edge.weight, direction='in')


In [10]:
edges_pdf = pd.DataFrame({'edge_id':edge_ids, 'level':levels, 'edge_y1':edge_y1s, 'edge_y2': edge_y2s, 'edge_weight': edge_weights})
edges_pdf.head()

,edge_id,level,edge_y1,edge_y2,edge_weight
0,edge_000,1.00,0,-30,30
1,edge_000,1.05,0,-30,30
2,edge_000,1.10,0,-30,30
3,edge_000,1.90,0,-30,30
4,edge_000,1.95,0,-30,30


In [11]:
def get_svg_altair_html(ap):
    html_container = io.StringIO()
    ap.save(html_container, embed_options={'renderer':'svg'}, format='html')
    html_container.seek(0)
    svg_altair_html = html_container.read()
    return HTML(svg_altair_html)


edge_highlighter = alt.selection_point(fields=['edge_id'], on='mouseover', empty=False)

edges_ap = (
    alt.Chart(edges_pdf)
        #.transform_filter(alt.datum.edge_id == 'edge_004')
        .transform_calculate(
            edge_weight_scaled='datum.edge_weight*1.4',
            edge_y='datum.edge_y1-datum.edge_weight/2'
        )
        .encode(
            x=alt.X('level:Q').axis(None),
            y=alt.Y('edge_y:Q').axis(None),
            #y2=alt.Y2('edge_y2:Q'),
            color=alt.value('orangered'),
            detail=alt.Detail('edge_id:O'),
            opacity=alt.condition(edge_highlighter, alt.value(0.8), alt.value(0.5)),
            size=alt.Size('edge_weight_scaled:Q').scale(None)
        )
        #.mark_area(tooltip=True, interpolate='monotone', point=True, line=False)
        .mark_line(tooltip=True, interpolate='monotone', cornerRadius=0, strokeCap='butt')
        .add_params(edge_highlighter)
        .properties(width=800, height=500)

)

nodes_ap = (
    alt.Chart(nodes_pdf)
        .encode(
            x=alt.X('level:Q').axis(None),
            y=alt.Y('node_y1:Q').axis(None),
            y2=alt.Y2('node_y2:Q'),
            tooltip=[
                alt.Tooltip('level:Q'),
                alt.Tooltip('weight:Q')
            ],
            color=alt.value('orangered'),
            opacity=alt.value(1)
        )
        .mark_bar(tooltip=True, size=10, stroke='black', strokeWidth=0.5)
)

ap = (edges_ap + nodes_ap).resolve_scale(size='independent').configure_view(strokeWidth=0)
#(edges_ap).configure_view(strokeWidth=0)
get_svg_altair_html(ap)